In [1]:
import pandas as pd
import numpy
import math
from collections import Counter

In [2]:
def truncate1(x):
    return math.trunc(x * 10) / 10

def truncate2(x):
    return math.trunc(x * 100) / 100

In [3]:
#define AQI calculator
def aqi_calc(x, col_name):
    
    classification = ""

    if col_name == 'PM1':
        x = truncate2(x)
        
        #PM1
        if 0 <= x <= 2.64:
            ind_hi = 50
            ind_low = 0
            bp_hi = 2.64
            bp_low = 0
            classification = "Good"
            
        if 2.65 <= x <= 7.78:
            ind_hi = 100
            ind_low = 51
            bp_hi = 7.78
            bp_low = 2.65
            classification = "Moderate"
            
        if 7.79 <= x <= 12.18:
            ind_hi = 150
            ind_low = 101
            bp_hi = 12.18
            bp_low = 7.79
            classification = "Unhealthy for Sensitive Groups"
            
        if 12.19 <= x <= 33.08:
            ind_hi = 200
            ind_low = 151
            bp_hi = 33.08
            bp_low = 12.19
            classification = "Unhealthy"
            
        if 33.09 <= x <= 55.08:
            ind_hi = 300
            ind_low = 201
            bp_hi = 55.08
            bp_low = 33.09
            classification = "Very Unhealthy"
            
        if 55.09 <= x <= 77.08:
            ind_hi = 400
            ind_low = 301
            bp_hi = 77.08
            bp_low = 55.09
            classification = "Hazardous"
            
        if 77.09 <= x:
            ind_hi = 500
            ind_low = 401
            bp_hi = 110.08
            bp_low = 77.09
            classification = "Hazardous"
    
    if col_name == 'PM2.5':
        x = truncate1(x)
        
        #PM2.5
        if 0 <= x <= 12:
            ind_hi = 50
            ind_low = 0
            bp_hi = 12
            bp_low = 0
            classification = "Good"
            
        if 12.1 <= x <= 35.4:
            ind_hi = 100
            ind_low = 51
            bp_hi = 35.4
            bp_low = 12.1
            classification = "Moderate"
            
        if 35.5 <= x <= 55.4:
            ind_hi = 150
            ind_low = 101
            bp_hi = 55.4
            bp_low = 35.5
            classification = "Unhealthy for Sensitive Groups"
            
        if 55.5 <= x <= 150.4:
            ind_hi = 200
            ind_low = 151
            bp_hi = 150.4
            bp_low = 55.5
            classification = "Unhealthy"
            
        if 150.5 <= x <= 250.4:
            ind_hi = 300
            ind_low = 201
            bp_hi = 250.4
            bp_low = 150.5
            classification = "Very Unhealthy"
            
        if 250.5 <= x <= 350.4:
            ind_hi = 400
            ind_low = 301
            bp_hi = 350.4
            bp_low = 255.5
            classification = "Hazardous"
            
        if 350.4 <= x:
            ind_hi = 500
            ind_low = 401
            bp_hi = 500.4
            bp_low = 350.5
            classification = "Hazardous"
    
    #PM10
    if col_name == 'PM10':
        x = math.trunc(x)
        
        if 0 <= x <= 54:
            ind_hi = 50
            ind_low = 0
            bp_hi = 54
            bp_low = 0
            classification = "Good"
            
        if 55 <= x <= 154:
            ind_hi = 100
            ind_low = 51
            bp_hi = 154
            bp_low = 55
            classification = "Moderate"
            
        if 155 <= x <= 254:
            ind_hi = 150
            ind_low = 101
            bp_hi = 254
            bp_low = 155
            classification = "Unhealthy for Sensitive Groups"
            
        if 255 <= x <= 354:
            ind_hi = 200
            ind_low = 151
            bp_hi = 354
            bp_low = 255
            classification = "Unhealthy"
            
        if 355 <= x <= 424:
            ind_hi = 300
            ind_low = 201
            bp_hi = 424
            bp_low = 355
            classification = "Very Unhealthy"
            
        if 425 <= x <= 504:
            ind_hi = 400
            ind_low = 301
            bp_hi = 504
            bp_low = 425
            classification = "Hazardous"
            
        if 505 <= x:
            ind_hi = 500
            ind_low = 401
            bp_hi = 604
            bp_low = 505
            classification = "Hazardous"
    
    aqi = ((ind_hi - ind_low)/(bp_hi - bp_low)) * (x - bp_low) + ind_low
    
    if aqi < 0:
        aqi = 1000
        classification = "Hazardous"
    
    return (aqi, classification)

In [4]:
def dataproc(year):
    
    extension = '.csv'
    file_name = year + extension
    
    #load dataframe
    data_frame = pd.read_csv('2011-2016.csv')
    
    data_frame = data_frame.drop(data_frame.index[data_frame.Year != int(year)])
    
    #drop rows with missing values
    missing = -999
    data_frame = data_frame.drop(data_frame.index[data_frame.PM10 == missing])
    data_frame = data_frame.drop(data_frame.index[data_frame['PM2.5'] == missing])
    data_frame = data_frame.drop(data_frame.index[data_frame.PM1 == missing])
    data_frame = data_frame.drop(data_frame.index[data_frame['Ambient Temperature'] == missing])
    data_frame = data_frame.drop(data_frame.index[data_frame['Relative Humidity'] == missing])
    
    #cut extreme values
    coarse_lower = data_frame.PM10.quantile(0.025)
    fine_lower = data_frame['PM2.5'].quantile(0.025)
    ultrafine_lower = data_frame.PM1.quantile(0.025)

    coarse_upper = data_frame.PM10.quantile(0.995)
    fine_upper = data_frame['PM2.5'].quantile(0.995)
    ultrafine_upper = data_frame.PM1.quantile(0.995)

    data_frame = data_frame.drop(data_frame.index[data_frame.PM10 < coarse_lower])
    data_frame = data_frame.drop(data_frame.index[data_frame.PM10 > coarse_upper])

    data_frame = data_frame.drop(data_frame.index[data_frame['PM2.5'] < fine_lower])
    data_frame = data_frame.drop(data_frame.index[data_frame['PM2.5'] > fine_upper])

    data_frame = data_frame.drop(data_frame.index[data_frame.PM1 < ultrafine_lower])
    data_frame = data_frame.drop(data_frame.index[data_frame.PM1 > ultrafine_upper])
    
    #calculate aqi + assign classification
    
    #add PM10 AQI col
    data_frame['PM10 AQI'] = ""
    data_frame['PM10 Classification'] = ""

    #add PM2.5 AQI col
    data_frame['PM2.5 AQI'] = ""
    data_frame['PM2.5 Classification'] = ""
    
    #add PM1 AQI col
    data_frame['PM1 AQI'] = ""
    data_frame['PM1 Classification'] = ""

    data_frame = data_frame.reset_index()


    pm_name = 'PM10'

    for index, row in data_frame.PM10.iteritems():
        pm_value = data_frame.iloc[index]['PM10']
        aqi = aqi_calc(pm_value, pm_name)
        data_frame.loc[index, 'PM10 AQI'] = aqi[0]
        data_frame.loc[index, 'PM10 Classification'] = aqi[1]

    pm_name = 'PM2.5'

    for index, row in data_frame['PM2.5'].iteritems():
        pm_value = data_frame.iloc[index]['PM2.5']
        aqi = aqi_calc(pm_value, pm_name)
        data_frame.loc[index, 'PM2.5 AQI'] = aqi[0]
        data_frame.loc[index, 'PM2.5 Classification'] = aqi[1]   
    

    pm_name = 'PM1'

    for index, row in data_frame.PM10.iteritems():
        pm_value = data_frame.iloc[index]['PM1']
        aqi = aqi_calc(pm_value, pm_name)
        data_frame.loc[index, 'PM1 AQI'] = aqi[0]
        data_frame.loc[index, 'PM1 Classification'] = aqi[1]
        
    data_frame.to_csv(file_name)
    data_frame
    
    
    file_name = year + ' normalised' + extension
    
    #normalise data and save
    
    #Encode days
    data_frame.Day = data_frame.Day.replace(to_replace = "Monday", value = 1)
    data_frame.Day = data_frame.Day.replace(to_replace = "Tuesday", value = 2)
    data_frame.Day = data_frame.Day.replace(to_replace = "Wednesday", value = 3)
    data_frame.Day = data_frame.Day.replace(to_replace = "Thursday", value = 4)
    data_frame.Day = data_frame.Day.replace(to_replace = "Friday", value = 5)
    data_frame.Day = data_frame.Day.replace(to_replace = "Saturday", value = 6)
    data_frame.Day = data_frame.Day.replace(to_replace = "Sunday", value = 7)

    #Encode classifications
    data_frame['PM10 Classification'] = data_frame.Day.replace(to_replace = "Good", value = 1)
    data_frame['PM10 Classification'] = data_frame.Day.replace(to_replace = "Moderate", value = 2)
    data_frame['PM10 Classification'] = data_frame.Day.replace(to_replace = "Unhealthy for Sensitive Groups", value = 3)
    data_frame['PM10 Classification'] = data_frame.Day.replace(to_replace = "Unhealthy", value = 4)
    data_frame['PM10 Classification'] = data_frame.Day.replace(to_replace = "Very Unhealthy", value = 5)
    data_frame['PM10 Classification'] = data_frame.Day.replace(to_replace = "Hazardous", value = 6)  

    data_frame['PM2.5 Classification'] = data_frame.Day.replace(to_replace = "Good", value = 1)
    data_frame['PM2.5 Classification'] = data_frame.Day.replace(to_replace = "Moderate", value = 2)
    data_frame['PM2.5 Classification'] = data_frame.Day.replace(to_replace = "Unhealthy for Sensitive Groups", value = 3)
    data_frame['PM2.5 Classification'] = data_frame.Day.replace(to_replace = "Unhealthy", value = 4)
    data_frame['PM2.5 Classification'] = data_frame.Day.replace(to_replace = "Very Unhealthy", value = 5)
    data_frame['PM2.5 Classification'] = data_frame.Day.replace(to_replace = "Hazardous", value = 6)  
    
    data_frame['PM1 Classification'] = data_frame.Day.replace(to_replace = "Good", value = 1)
    data_frame['PM1 Classification'] = data_frame.Day.replace(to_replace = "Moderate", value = 2)
    data_frame['PM1 Classification'] = data_frame.Day.replace(to_replace = "Unhealthy for Sensitive Groups", value = 3)
    data_frame['PM1 Classification'] = data_frame.Day.replace(to_replace = "Unhealthy", value = 4)
    data_frame['PM1 Classification'] = data_frame.Day.replace(to_replace = "Very Unhealthy", value = 5)
    data_frame['PM1 Classification'] = data_frame.Day.replace(to_replace = "Hazardous", value = 6)  

    #normalisation
    data_frame.PM10 = (data_frame.PM10 - data_frame.PM10.min())/(data_frame.PM10.max()-data_frame.PM10.min())
    data_frame['PM2.5'] = (data_frame['PM2.5'] - data_frame['PM2.5'].min())/(data_frame['PM2.5'].max()-data_frame['PM2.5'].min())
    data_frame.PM1 = (data_frame.PM1 - data_frame.PM1.min())/(data_frame.PM1.max()-data_frame.PM1.min())
    
    data_frame['PM10 AQI'] = (data_frame['PM10 AQI'] - data_frame['PM10 AQI'].min())/(data_frame['PM10 AQI'].max()-data_frame['PM10 AQI'].min())
    data_frame['PM2.5 AQI'] = (data_frame['PM2.5 AQI'] - data_frame['PM2.5 AQI'].min())/(data_frame['PM2.5 AQI'].max()-data_frame['PM2.5 AQI'].min())
    data_frame['PM1 AQI'] = (data_frame['PM1 AQI'] - data_frame['PM1 AQI'].min())/(data_frame['PM1 AQI'].max()-data_frame['PM1 AQI'].min())
    
    data_frame['Relative Humidity'] = (data_frame['Relative Humidity'] - data_frame['Relative Humidity'].min())/(data_frame['Relative Humidity'].max()-data_frame['Relative Humidity'].min())
    data_frame['Ambient Temperature'] = (data_frame['Ambient Temperature'] - data_frame['Ambient Temperature'].min())/(data_frame['Ambient Temperature'].max()-data_frame['Ambient Temperature'].min())
    data_frame.Precipitation = (data_frame.Precipitation - data_frame.Precipitation.min())/(data_frame.Precipitation.max()-data_frame.Precipitation.min())
    #data_frame.drop(columns=['PM10 AQI', 'PM2.5 AQI', 'PM10 Classification', 'PM2.5 Classification'])
    
    data_frame.to_csv(file_name)

In [5]:
dataproc('2011')
dataproc('2012')
dataproc('2013')
dataproc('2014')
dataproc('2015')
dataproc('2016')